In [1]:
import os
import logging
import configparser
import json

#####################   ML
from ML.ml_utils import ML_utils 
from ML.ml_utils import ML_main
import tensorflow as tf
import keras as keras
import datetime
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()
from data.plot_data import plotMaskedImage
from data.plot_data import plotPred
import data.plot_data as pd

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import numpy as np

import seaborn as sns

Segmentation Models: using `keras` framework.


In [2]:
ml = ML_utils()

#val_gen = ml.DataGenerator(val_images, val_masks)

model = keras.models.load_model("/localhome/studenter/mikaellv/Project/ML/models/test_model_4")
model.summary()

### test img
img_path = '/localhome/studenter/renatask/Project/data/datasets/data_medium/test/images'
mask_path = '/localhome/studenter/renatask/Project/data/datasets/data_medium/test/masks'

classes = {
    1: 'Water',
    0: 'not_water',
}

def getSegmentationArr(image, classes):
    width=image.shape[1]
    height=image.shape[0]
    seg_labels = np.zeros((height, width, classes))
    img = image[:, :, 0]

    for c in range(classes):
        seg_labels[:, :, c] = (img == c ).astype(int)
    return seg_labels

2022-03-24 18:01:48.069112: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 18:01:48.096767: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 18:01:48.097501: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 18:01:48.098667: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 bn_data (BatchNormalization)   (None, None, None,   9           ['data[0][0]']                   
                                3)                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, None, None,   0          ['bn_data[0][0]']                
                                3)                                                          

In [3]:
batch_size=32
nr_classes = len(classes)
files = [f for f in os.listdir(img_path) if not f.startswith('.')] #os.listdir(path+'/images')
for i in range(0, 1, batch_size): #for i in range(0, len(files), batch_size):
    batch_files = files[i : i+batch_size]
    imgs=[]
    segs=[]
    for file in batch_files:
        if not file.startswith('.'):
            image, mask = ml.LoadImage(file=file,image_path=img_path, mask_path=mask_path)
            mask_binned = ml.bin_image(mask)
            label = getSegmentationArr(mask_binned, nr_classes)

            H_32 = int(image.shape[0]/32)*32
            W_32 = int(image.shape[1]/32)*32

            image = image[0:H_32,0: W_32,:]
            label = label[0:H_32, 0:W_32,:]

            imgs.append(image[:,:,0:3])  #uses the first 3 bands 
            segs.append(label)

imgs = np.array(imgs)
segs = np.array(segs)
preds = []

for i in range(len(imgs)):
    img = np.expand_dims(imgs[i], axis=0)
    pred = model.predict(img)
    preds.append(pred)


2022-03-24 15:22:58.847287: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8301
2022-03-24 15:22:59.671526: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-03-24 15:22:59.691400: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-03-24 15:22:59.691434: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2022-03-24 15:22:59.691541: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be 

In [4]:
from matplotlib import pyplot as plt
def test_plotPred(image, mask, pred, name):
    #for i in image:
    #    for e in i:
    #        for a in range(len(e)):
    #            if math.isnan(e[a]): e[a]=0
    #image = scale(image)
    #print(image[1])
    fig, axs = plt.subplots(1, 3, figsize=(25,25))
    plt.tight_layout()
    #axs[0].imshow(10*np.log10(image[:, :, 0]), cmap='ocean') #image[:, :, 1]
    axs[0].imshow(image[:, :, 0], cmap='PuBuGn_r') #image[:, :, 1]
    #axs[0].hist(image.ravel(), bins=256, range=(0.0, 1.0), fc='k', ec='k')
    #rasterio.plot.show(image, ax=axs[0], adjust='linear')
    axs[0].set_title('Original Image')
    axs[2].imshow(mask[:, :, 1])
    axs[2].set_title('Segmentation Mask')
    #axs[1].imshow(pred[:, :, 1]) # use this line if argmax is not applied 
    axs[1].imshow(pred) # use this line if argmax is applied 
    #axs[1].imshow(10*np.log10(image[:, :, 0]), cmap='ocean', alpha=0.6)
    #rasterio.plot.show(image, ax=axs[1], adjust=False)
    #axs[1].imshow(mask, alpha=0.5)
    axs[1].set_title('Predicted mask')
    plt.savefig(name + '.png')

In [5]:
pred1 = np.rollaxis(preds[0],0,3)

test_plotPred(imgs[0], segs[0], np.argmax(pred1, axis=-1), 'test')

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-mikaellv'
libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


In [9]:
for i in range(len(os.listdir(img_path))):



    
    pred1 = np.rollaxis(preds[0],0,3)
    test_plotPred(imgs[0], segs[0], np.argmax(pred1, axis=-1), '/localhome/studenter/mikaellv/Project/predictions_test/'+str(i))


/tmp/ipykernel_2149886/1076740037.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(1, 3, figsize=(25,25))


In [ ]:
nr_classes = len(classes)
files = [f for f in os.listdir(img_path) if not f.startswith('.')] #os.listdir(path+'/images')
imgs=[]
segs=[]
for file in files:
    if not file.startswith('.'):
        image, mask = ml.LoadImage(file=file,image_path=img_path, mask_path=mask_path)
        mask_binned = ml.bin_image(mask)
        label = getSegmentationArr(mask_binned, nr_classes)

        H_32 = int(image.shape[0]/32)*32
        W_32 = int(image.shape[1]/32)*32

        image = image[0:H_32,0: W_32,:]
        label = label[0:H_32, 0:W_32,:]

        imgs.append(image[:,:,0:3])  #uses the first 3 bands 
        segs.append(label)

imgs = np.array(imgs)
segs = np.array(segs)
preds = []

for i in range(len(imgs)):
    img = np.expand_dims(imgs[i], axis=0)
    pred = model.predict(img)
    preds.append(pred)
    pred1 = np.rollaxis(preds[i],0,3)
    #test_plotPred(imgs[i], segs[i], np.argmax(pred1, axis=-1), '/localhome/studenter/renatask/Project/predictions_test/'+str(i))


/tmp/ipykernel_2093013/841314768.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  imgs = np.array(imgs)
/tmp/ipykernel_2093013/841314768.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  segs = np.array(segs)


In [ ]:
preds = np.array(preds)

print(f"segmentation shape: {segs.shape}")
print(f"predictions shape: {preds.shape}")

pred1D = preds.reshape(-1)
segs1D = segs.reshape(-1)

print(f"segmentation 1d: {segs1D.shape}")
print(f"predictions 1d: {pred1D.shape}")

print(f"Confusion matrix: \n {tf.math.confusion_matrix(segs1D, pred1D, num_classes=ml.N_CLASSES)}")

precision = precision_score(segs1D, pred1D, average='weighted')
recall = recall_score(segs1D, pred1D, average='weighted')
print(f'Precision score: {precision}')
print(f'Recall score: {recall}')
print(f'F1 score: {(2*precision*recall)/(recall+precision)}')
print(f"Confusion matrix: \n {confusion_matrix(segs1D, pred1D)}")
f1=f1_score(segs1D, pred1D, average='weighted')
print(f'F1 score: {f1}')

/tmp/ipykernel_2093013/328652661.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  preds = np.array(preds)


ValueError: could not broadcast input array from shape (1632,1632,2) into shape (1,)